<a href="https://colab.research.google.com/github/stellagerantoni/LatentCfMultivariate/blob/main/FingerMovements_KDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/stellagerantoni/LatentCfMultivariate

Cloning into 'LatentCfMultivariate'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 189 (delta 65), reused 36 (delta 36), pack-reused 105
Receiving objects: 100% (189/189), 12.36 MiB | 7.96 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [2]:
!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!pip install aeon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.6 MB/s eta 0:00:00


In [4]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals
%cd '/content/LatentCfMultivariate'
from _guided import ModifiedLatentCF
from help_functions import *
from keras_models import *

/content/LatentCfMultivariate


In [5]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

In [6]:
def load_dataset(dataset):
  X, y = load_classification(dataset)
  if dataset == 'FingerMovements':
    pos = 'left'
    neg = 'right'


  print(" Shape of X = ", X.shape)
  print(" Shape of y = ", y.shape)
  #print(" Meta data = ", meta_data)
  # Convert positive and negative labels to 1 and 0
  pos_label, neg_label = 1, 0
  if pos != pos_label:
      y[y==pos] = pos_label # convert/normalize positive label to 1
  if neg != neg_label:
      y[y==neg] = neg_label # convert negative label to 0

  y = y.astype(int)
  print(f"\n X[:1] = \n{X[:1]}")
  return X,y,pos_label, neg_label

# **ACTUALL CODE**


In [14]:
RANDOM_STATE = 39
X,y,pos_label,neg_label = load_dataset('FingerMovements')
X = X.transpose(0,2,1)
print(f'shape of X = {X.shape}')
print(f'shape of y = {y.shape}')
#print(f'data imformation = {data_information}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f'shape of X train = {X_train.shape}')
print(f'shape of y train = {y_train.shape}')

 Shape of X =  (416, 28, 50)
 Shape of y =  (416,)

 X[:1] = 
[[[41.8 44.8 47.1 ... 69.8 72.6 76.1]
  [55.2 53.8 59.9 ... 17.5 28.  12.1]
  [-8.6 -3.6 14.4 ... 23.3 35.9 23.2]
  ...
  [16.9 24.5 24.5 ... 51.9 59.6 57.3]
  [42.2 35.  41.7 ... 51.5 58.5 46.9]
  [13.  26.6 52.5 ... -3.5 -3.2 -2.6]]]
shape of X = (416, 50, 28)
shape of y = (416,)
shape of X train = (332, 50, 28)
shape of y train = (332,)


In [15]:
#Upsample the minority class

unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'before: {class_counts}')
X_train,y_train = upsample_minority_multivariate(X_train,y_train)
X,y = upsample_minority_multivariate(X, y)
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'after: {class_counts}')

before: [166 166]
after: [166 166]


In [16]:
#Processing and Padding all our data
#Padding needed for autoencoder

n_training,n_timesteps, n_features= X_train.shape

X, trained_scaler =  normalize_multivariate(data=X, n_timesteps=n_timesteps, n_features = n_features)
X_train_processed, trained_scaler =  normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X, padding_size = conditional_pad_multivariate(X)
X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[1]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

#check the processing (0,1) min should be min 0 and max should be max 1
print(f"\nmin value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

#check that padding paddes the right dimention
print(f"\nX_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")


Data pre-processed, original #timesteps=50, padded #timesteps=52.

min value = -132.1, max value = 205.1
min value normalized = 0.0, max value normalized= 1.0

X_train.shape = (332, 50, 28)
X_train_processed_padded.shape = (332, 52, 28)


In [17]:
#splitting the dataset

from sklearn.model_selection import train_test_split
X_train,X_validation, y_train, y_validation = train_test_split(X_train_processed_padded, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train)

In [18]:
#Getting the two forms of labels needed
#-the y_classes (1,0,1,0,...)
#-the y (one hot encoded)

print(f'X_train = {X_train.shape}')
print(f'X_validation = {X_validation.shape}')
print(f'X_test = {X_test.shape}')

y_classes = y
y_train_classes = y_train
y_validation_classes = y_validation
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, len(np.unique(y)))
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_validation = to_categorical(y_validation, len(np.unique(y_validation)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

print(f'\ny_train_classes = {y_train_classes.shape}, y_validation_classes = {y_validation_classes.shape}, y_test_classes = {y_test_classes.shape}')
print(f'y_train = {y_train.shape}, y_validation = {y_validation.shape}, y_test= {y_test.shape}')

X_train = (265, 52, 28)
X_validation = (67, 52, 28)
X_test = (84, 50, 28)

y_train_classes = (265,), y_validation_classes = (67,), y_test_classes = (84,)
y_train = (265, 2), y_validation = (67, 2), y_test= (84, 2)


In [22]:


# ## LatentCF++ models
# reset seeds for numpy, tensorflow, python random package and python environment seed
reset_seeds()
###############################################
# ### 1dCNN classifier

cnnClassifier = Classifier(
    n_timesteps_padded, n_features, n_output=2, add_dense_layer = False
)

optimizer = keras.optimizers.Adam(lr=0.001)
cnnClassifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=15, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM-FCN classifier:")
classifier_history = cnnClassifier.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=12,
    shuffle=True,
    verbose=True,
    validation_data=(X_validation, y_validation),
    callbacks=[early_stopping_accuracy],
)

y_pred = cnnClassifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM-FCN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:1", "True:0"],
    columns=["Pred:1", "Pred:0"],
)
print(confusion_matrix_df)


Training log for LSTM-FCN classifier:
Epoch 1/150
23/23 [==============================] - 4s 42ms/step - loss: 0.8307 - accuracy: 0.5132 - val_loss: 0.7577 - val_accuracy: 0.4925
Epoch 2/150
23/23 [==============================] - 0s 10ms/step - loss: 0.6913 - accuracy: 0.6226 - val_loss: 0.7282 - val_accuracy: 0.5224
Epoch 3/150
23/23 [==============================] - 0s 8ms/step - loss: 0.6332 - accuracy: 0.7132 - val_loss: 0.7123 - val_accuracy: 0.6567
Epoch 4/150
23/23 [==============================] - 0s 7ms/step - loss: 0.7008 - accuracy: 0.6189 - val_loss: 0.7063 - val_accuracy: 0.6418
Epoch 5/150
23/23 [==============================] - 0s 9ms/step - loss: 0.5797 - accuracy: 0.7585 - val_loss: 0.7009 - val_accuracy: 0.6567
Epoch 6/150
23/23 [==============================] - 0s 9ms/step - loss: 0.5776 - accuracy: 0.7736 - val_loss: 0.7315 - val_accuracy: 0.4925
Epoch 7/150
23/23 [==============================] - 0s 13ms/step - loss: 0.5878 - accuracy: 0.7321 - val_loss: 0.

In [23]:
reset_seeds()

# ### 1dCNN autoencoder
autoencoder = Autoencoder( n_timesteps_padded,n_features,32)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoder.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoder.fit(
    X_train,
    X_train,
    epochs=50,
    batch_size=12,
    shuffle=True,
    verbose=2,
    validation_data=(X_validation, X_validation),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")


(None, 52, 28)
(None, 52, 32)
(None, 26, 32)
(None, 26, 16)
(None, 13, 16)
(None, 13, 16)
(None, 26, 16)
(None, 26, 32)
(None, 52, 32)
(None, 52, 28)
Training log for 1dCNN autoencoder:
Epoch 1/50
23/23 - 2s - loss: 0.1518 - val_loss: 0.0500 - 2s/epoch - 66ms/step
Epoch 2/50
23/23 - 0s - loss: 0.0250 - val_loss: 0.0140 - 133ms/epoch - 6ms/step
Epoch 3/50
23/23 - 0s - loss: 0.0131 - val_loss: 0.0112 - 129ms/epoch - 6ms/step
Epoch 4/50
23/23 - 0s - loss: 0.0112 - val_loss: 0.0099 - 129ms/epoch - 6ms/step
Epoch 5/50
23/23 - 0s - loss: 0.0101 - val_loss: 0.0092 - 139ms/epoch - 6ms/step
Epoch 6/50
23/23 - 0s - loss: 0.0094 - val_loss: 0.0083 - 128ms/epoch - 6ms/step
Epoch 7/50
23/23 - 0s - loss: 0.0087 - val_loss: 0.0077 - 130ms/epoch - 6ms/step
Epoch 8/50
23/23 - 0s - loss: 0.0080 - val_loss: 0.0071 - 126ms/epoch - 5ms/step
Epoch 9/50
23/23 - 0s - loss: 0.0074 - val_loss: 0.0066 - 136ms/epoch - 6ms/step
Epoch 10/50
23/23 - 0s - loss: 0.0068 - val_loss: 0.0060 - 124ms/epoch - 5ms/step
Epoch

In [25]:
#Gettting the Global weights, needed for counterfactuals

from _guided import get_global_weights
from help_functions import evaluate
pos_label = 1
neg_label = 0

step_weights = get_global_weights(
        X,
        y_classes,
        cnnClassifier,
        n_timesteps= n_timesteps,
        n_features=n_features,
        random_state=RANDOM_STATE,
)


13/13 [==============================] - 0s 3ms/step


In [26]:
step_weights

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [34]:
# @title Default title text
import warnings

import numpy as np
import tensorflow as tf
from tensorflow import keras

from wildboar.explain import IntervalImportance
#from LIMESegment.Utils.explanations import LIMESegment


class ModifiedLatentCF:
    """Explanations by generating a counterfacutal sample in the latent space of
    any autoencoder.

    References
    ----------
    Learning Time Series Counterfactuals via Latent Space Representations,
    Wang, Z., Samsten, I., Mochaourab, R., Papapetrou, P., 2021.
    in: International Conference on Discovery Science, pp. 369–384. https://doi.org/10.1007/978-3-030-88942-5_29
    """

    def __init__(
        self,
        probability=0.5,
        *,
        tolerance=1e-6,
        max_iter=100,
        optimizer=None,
        autoencoder=None,
        margin_weight=1.0,  # weighted_steps_weight = 1 - pred_margin_weight
        random_state=None,
        bandwidth,
        weighted_steps_weight,
        data,
        step_weights
    ):
        self.optimizer_ = (
            tf.optimizers.Adam(learning_rate=1e-4) if optimizer is None else optimizer
        )
        #self.x_axis_eights = x_axis_eights
        #self.y_axis_eights = y_axis_eights
        self.data = data
        self.mse_loss_ = keras.losses.MeanSquaredError()
        self.probability_ = tf.constant([probability])
        self.tolerance_ = tf.constant(tolerance)
        self.max_iter = max_iter
        self.autoencoder = autoencoder
        self.random_state = random_state
        self.weighted_steps_weight = weighted_steps_weight
        self.step_weights = step_weights

        # Weights of the different loss components
        self.margin_weight = margin_weight
        self.kde_weight = tf.cast(1 - self.margin_weight-self.weighted_steps_weight,tf.float32)
        if (self.weighted_steps_weight+self.margin_weight)>1.0:
           raise ValueError("(weighted_steps_weight + margin_weight) should be less that 1.0")

        self.bandwidth = bandwidth

    def fit(self, model):
        """Fit a new counterfactual explainer to the model

        Paramaters
        ----------

        model : keras.Model
            The model
        """
        if self.autoencoder:
            (
                encode_input,
                encode_output,
                decode_input,
                decode_output,
            ) = extract_encoder_decoder(self.autoencoder)
            self.decoder_ = keras.Model(inputs=decode_input, outputs=decode_output)
            self.encoder_ = keras.Model(inputs=encode_input, outputs=encode_output)
        else:
            self.decoder_ = None
            self.encoder_ = None
        self.model_ = model
        return self

    def predict(self, x):
        """Compute the difference between the desired and actual probability

        Parameters
        ---------
        x : Variable
            Variable of the sample
        """
        if self.autoencoder is None:
            z = x
        else:
            z = self.decoder_(x)

        return self.model_(z)

    # The "pred_margin_loss" is designed to measure the prediction probability to the desired decision boundary
    def pred_margin_mse(self, prediction):
        return self.mse_loss_(self.probability_, prediction)

    # An auxiliary MAE loss function to measure the proximity with step_weights
    def weighted_mae(self, original_sample, cf_sample, step_weights):
        return tf.math.reduce_mean(
            tf.math.multiply(tf.math.abs(original_sample - cf_sample), step_weights)
        )

    # An auxiliary normalized L2 loss function to measure the proximity with step_weights
    def weighted_normalized_l2(self, original_sample, cf_sample, step_weights):
        var_diff = tf.math.reduce_variance(original_sample - cf_sample)
        var_orig = tf.math.reduce_variance(original_sample)
        var_cf = tf.math.reduce_variance(cf_sample)

        normalized_l2 = 0.5 * var_diff / (var_orig + var_cf)
        return tf.math.reduce_mean(
            tf.math.multiply(
                normalized_l2,
                step_weights,
            )
        )
    #x_axis_eights*self.step_weights_x,use_scotts_rule=False,use_silvermans_rule=True,manual_bandwidth = 0.1
    def train_gaussian_kde(self, data, use_scotts_rule,use_silvermans_rule, manual_bandwidth ):
      """
      Train a Gaussian KDE on the provided data.

      :param data: Multivariate data points used for KDE (2D Tensor).
      :param bandwidth: The bandwidth of the kernel (float).
      :return: A function that represents the trained KDE.
      """

      n = tf.cast(tf.shape(data)[0], tf.float32)
      d = tf.cast(tf.shape(data)[1], tf.float32)
      def kde_fn( x_points):
        """
        Compute the density estimation for given points using the trained KDE.

        :param x_points: Points where the density should be estimated (2D Tensor).
        :return: Density estimates (Tensor).
        """
        d = tf.cast(tf.shape(data)[1], tf.float32)

        data_exp =  tf.expand_dims(data, axis=0)
        print(f'x_points shape = {x_points.shape}')
        x_points_exp = tf.reshape(x_points, (tf.shape(x_points)[0], 28, 1))

        x_points_exp = tf.expand_dims(x_points_exp, axis=1)
        if use_scotts_rule:
          sigma = tf.math.reduce_std(data)
          sigma = tf.cast(sigma,tf.float32)
          bandwidth = n ** (-1.0 / (d + 4)) * sigma
        elif use_silvermans_rule:
          sigma = tf.math.reduce_std(data)
          bandwidth = (4 * sigma**5 / (3 * n)) ** (1/5)
        else:
          if manual_bandwidth is None:
            raise ValueError("Manual bandwidth must be provided if not using Scott's Rule.")
          bandwidth = manual_bandwidth

        bandwidth = tf.cast(bandwidth, tf.float32)
        x_points_exp = tf.cast(x_points_exp,tf.float32)
        data_exp= tf.cast(data_exp, tf.float32)
        diff = x_points_exp - data_exp
        norm = tf.reduce_sum(diff ** 2, axis=2)
        kernel_val = tf.exp(-norm / (2.0 * bandwidth ** 2))


        d = tf.constant(d,tf.float32)
        density = tf.reduce_mean(kernel_val, axis=1) / (bandwidth * tf.sqrt(2.0 * np.pi * d))
        return density

      return kde_fn

    def gaussian_kde_logpdf(self, kde_fn, x_points):
      """
      Evaluate the logpdf of the given points using the trained KDE function.

      :param kde_fn: Trained KDE function.
      :param x_points: Points to evaluate the logpdf (2D Tensor).
      :return: Log of the density estimates (Tensor).
      """
      density = kde_fn(x_points)
      return tf.math.log(density)


    # additional input of step_weights
    def compute_loss(self,original_sample, z_search, target_label, kde_stop):
        loss = tf.zeros(shape=())
        decoded = self.decoder_(z_search) if self.autoencoder is not None else z_search
        pred = self.model_(decoded)[:, target_label]

        #margin loss (y-τ)
        margin_loss = self.pred_margin_mse(pred)
        loss += self.margin_weight * margin_loss

        kde_diffrences = []
        for dimention in range(decoded.shape[2]):
          data_dimention = self.data[:,:,dimention]
          data_dimention = data_dimention[:,:,np.newaxis]
          kde = self.train_gaussian_kde(data = data_dimention,use_scotts_rule=False,use_silvermans_rule=False,manual_bandwidth = self.bandwidth)
          mean_log_likelihood = tf.cast(tf.reduce_mean(self.gaussian_kde_logpdf(kde, x_points = (data_dimention))),tf.float32)

          decoded_dimention = decoded[:,:,dimention]
          decoded_dimention = decoded_dimention[:,:,np.newaxis]
          log_likelihood_of_sample = tf.cast(self.gaussian_kde_logpdf(kde, x_points = (decoded_dimention)),tf.float32)

          kde_loss = tf.cast((mean_log_likelihood - log_likelihood_of_sample),tf.float32)
          kde_diffrences.append(kde_loss)
        kde_total_loss = tf.math.add_n(kde_diffrences)
        kde_total_loss = tf.math.abs(kde_total_loss)
        print(kde_total_loss)
        loss +=self.kde_weight *kde_total_loss


        weighted_steps_loss = self.weighted_mae(
            original_sample=tf.cast(original_sample, dtype=tf.float32),
            cf_sample=tf.cast(decoded, dtype=tf.float32),
            step_weights=tf.cast(step_weights, tf.float32),
        )
        loss += self.weighted_steps_weight * weighted_steps_loss

        return loss, margin_loss, kde_loss, kde_stop

    # TODO: compatible with the counterfactuals of wildboar
    #       i.e., define the desired output target per label


    def transform(self, x, pred_labels):
        """Generate counterfactual explanations

        x : array-like of shape [n_samples, n_timestep, n_dims]
            The samples
        """

        result_samples = np.empty(x.shape)
        losses = np.empty(x.shape[0])
        # `weights_all` needed for debugging
        weights_all = np.empty((x.shape[0], 1, x.shape[1], x.shape[2]))
        for i in range(x.shape[0]):
            print(f"sample {i+1} is beeing transformed.")
            kde_stop = False

            x_sample, loss = self._transform_sample(
                x[np.newaxis, i], pred_labels[i],kde_stop
            )

            result_samples[i] = x_sample
            losses[i] = loss
            weights_all[i] = step_weights

        print(f"{i+1} samples been transformed, in total.")

        return result_samples, losses, weights_all

    def _transform_sample(self, x, pred_label,kde_stop):
        """Generate counterfactual explanations(z))"""
        # TODO: check_is_fitted(self)
        if self.autoencoder is not None:
            z = tf.Variable(self.encoder_(x))
        else:
            z = tf.Variable(x, dtype=tf.float32)

        it = 0
        target_label = 1 - pred_label  # for binary classification

        #def compute_loss(self,original_sample, z_search, target_label)
        with tf.GradientTape(persistent = True) as tape:
            loss, pred_margin_loss, kde_loss, kde_stop = self.compute_loss(
                x, z,  target_label,kde_stop
            )
        if self.autoencoder is not None:
            pred = self.model_(self.decoder_(z))
        else:
            pred = self.model_(z)


        while (
            (loss > self.tolerance_
            and pred[:, target_label] < self.probability_)
            and (it < self.max_iter if self.max_iter else True)
            ) :
            # Get gradients of loss wrt the sample
            grads = tape.gradient(loss, z)
            # Update the weights of the sample
            self.optimizer_.apply_gradients([(grads, z)])
            del tape

            #self,original_sample, z_search, target_label
            with tf.GradientTape(persistent=True) as tape:
                loss, pred_margin_loss, kde_loss, kde_stop = self.compute_loss(
                    x, z,  target_label,kde_stop
                )

            kde_loss = tf.convert_to_tensor(kde_loss, dtype=tf.float32)
            grads = tape.gradient(loss, z)
            # Optionally, compute and log gradients for individual components
            kde_grads = tape.gradient(kde_loss, z)
            margin_grads = tape.gradient(pred_margin_loss, z)


            it += 1
            if it % 50 == 0:
              print(f'Currently on iteration: {it}.')
            #   # print(f'kde loss : {kde_loss}')
            #   # print(f'margin loss : {pred_margin_loss}')
            #   # print(f'loss : {loss}')

            #   if kde_grads is  None:
            #     print("KDE Gradients are None.")
            if self.autoencoder is not None:
                pred = self.model_(self.decoder_(z))
            else:
                pred = self.model_(z)

        # # uncomment for debug
        # print(
        #     f"current loss: {loss}, pred_margin_loss: {pred_margin_loss}, weighted_steps_loss: {weighted_steps_loss}, pred prob:{pred}, iter: {it}. \n"
        # )

        res = z.numpy() if self.autoencoder is None else self.decoder_(z).numpy()
        return res, float(loss)




In [35]:
reset_seeds()

cf_model = ModifiedLatentCF(
    probability=0.9,
    tolerance=1e-6,
    max_iter=500,
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    autoencoder = autoencoder,
    margin_weight=0.6,
    random_state= RANDOM_STATE,
    bandwidth = 0.5,
    weighted_steps_weight = 0.2,
    data = X_train[y_train_classes == 1],
    step_weights = step_weights
    )
cf_model.fit(cnnClassifier)


y_neg = y_classes[y_classes == 0][10:15]
X_neg = X[y_classes == 0][10:15]

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    cf_embeddings, losses, weights = cf_model.transform(x = X_neg,pred_labels = y_neg)


sample 1 is beeing transformed.
x_points shape = (132, 52, 1)


InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 6864 values, but the requested shape has 3696 [Op:Reshape]

In [ ]:
#with learning rate = 0.001, 500 iterations, bandwidth = 0.1,margin_weight=0.6,weighted_steps_weight = 0.2,kse_weight = 0.2
for i in range(5):
  i=i+30
  print("actual")
  visualise_digit(X_neg,y_neg,i,figsize = (5,5))
  print("counterfactual")
  visualise_digit(cf_embeddings,y_neg,i,figsize = (5,5))


In [ ]:
#Validity
cf_pred = cnnClassifier.predict(cf_embeddings)[:, 1] # predicted probabilities of CFs
cf_pred_labels = cf_pred
for idx in range(cf_pred_labels.shape[0]):
  if cf_pred_labels[idx] > 0.5:
    cf_pred_labels[idx] = 1
  else:
    cf_pred_labels[idx] = 0


print(f'Transformation_finished with validity_score = {validity_score(y_neg,cf_pred_labels)}')

In [ ]:
#Calculating proximity withour absolute
from tensorflow.keras.losses import MeanSquaredError
total_mse = 0
probability = 0.9
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    mse = MeanSquaredError()
    mse_dist = mse(probability, prediction).numpy()
    total_mse +=mse_dist
mean_mse = total_mse /cf_embeddings.shape[0]
print(f"The Mean MSE of the data is: {mean_mse} ")

In [ ]:
print(f"The Mean MSE of the data is: {mean_mse} ")

In [ ]:
#Calculating proximity with absolute
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.9
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = abs(prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]
print(f"The Mean MSE of the data is: {mean_mse} ")

In [ ]:
print(f"The Mean MSE of the data is: {mean_mse} ")

In [ ]:
#Proximity
def euclidean_distance(X, cf_samples):
    paired_distances = np.linalg.norm(X - cf_samples, axis=1)
    return np.mean(paired_distances)
euclidean_distance(X_neg, cf_embeddings)

In [ ]:
from scipy.stats import gaussian_kde

diffrences_from_abnormal = []
diffrences_from_normal = []
for dimention in range(cf_embeddings.shape[2]):


  abnormal_data = X[y_classes == 1][:,:,dimention]
  normal_data = X[y_classes == 0][:,:,dimention]
  counterf_data = cf_embeddings[:,:,dimention]

  #get the kernel for every dimention of the trained
  kernel = gaussian_kde(abnormal_data.T,bw_method=None)

  #get all the log likelihoods
  log_likelihood_abnormal = np.mean(kernel.logpdf(abnormal_data.T))
  log_likelihood_normal = np.mean(kernel.logpdf(normal_data.T))
  log_likelihood_counterfactual = np.mean(kernel.logpdf(counterf_data.T))

  #get the diffrences from the counterfactuals
  diff_from_abnormal = abs(log_likelihood_counterfactual-log_likelihood_abnormal)
  diffrences_from_abnormal.append(diff_from_abnormal)

  diff_from_normal = abs(log_likelihood_counterfactual-log_likelihood_normal)
  diffrences_from_normal.append(diff_from_normal)


In [ ]:
diffrences_from_normal

In [ ]:
diffrences_from_abnormal